# Marine Ice Sheet Stability Demo

This is a demonstration of some models for the evolution of marine ice sheets. This was modified for the Princeton AOS workshop 2021. Last updated by Alex Robel on July 20, 2021.

## Load Packages

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from scipy import optimize

## Define simple model with plotting

In [33]:
def plotmodel(accum,Linit):
    #other parameters
    rho_i = 917    #ice density
    rho_w = 1028   #water density
    g = 9.81       #gravity (in m/s^2)
    beta = 4       #GL flux exponent

    #time stepping
    tf = 5000       #length of simulation
    nt = 5000       #number of time steps
    dt = tf/nt     #length of time step
    L = np.zeros([nt+1]) #pre-allocate volume vector
    L[0] = Linit     #initial condition
    
    omega = 5e-7   #GL flux coefficient
    #run model for grounding line Evolution
    for t in range(nt):
        b = -(729 - 2184.8*(L[t]/750e3)**2 + 1031.72*(L[t]/750e3)**4 - 151.72*(L[t]/750e3)**6)   #calculate bed depth at current grounding line position from MISMIP topography
        hg = (rho_w/rho_i)*b      #calculate flotation thickness at GL
        Q_g = omega*(hg**beta)      #calculate GL flux
  
        dL_dt = (accum*L[t] - Q_g)/hg #calculate RHS of model
    
        L[t+1] = L[t] + dL_dt*dt       #use forward euler to advance model
    
    #calculate some diagnostics
    time = np.linspace(0,tf,nt+1)
    x = np.linspace(400.0,1500.0,1000)
    bx = (729 - 2184.8*(x/750)**2 + 1031.72*(x/750)**4 - 151.72*(x/750)**6)
    binit = (729 - 2184.8*(L[0]/750e3)**2 + 1031.72*(L[0]/750e3)**4 - 151.72*(L[0]/750e3)**6)
    bend = (729 - 2184.8*(L[-1]/750e3)**2 + 1031.72*(L[-1]/750e3)**4 - 151.72*(L[-1]/750e3)**6)
    hgs = -(rho_w/rho_i)*(729 - 2184.8*(x/750)**2 + 1031.72*(x/750)**4 - 151.72*(x/750)**6)
    hgsl = -(rho_w/rho_i)*(729 - 2184.8*(L/750e3)**2 + 1031.72*(L/750e3)**4 - 151.72*(L/750e3)**6)

    #plot time evolution of grounding line
    fig, (ax1, ax2, ax3) = plt.subplots(3,figsize=(10,10))

    ax1.set_xlabel('x (km)')  # we already handled the x-label with ax1
    ax1.set_ylabel('Bed topography (m)')  # we already handled the x-label with ax1
    ax1.plot(x, bx)
    ax1.plot(L[0]/1e3,binit,'bs', markersize=12)
    ax1.plot(L[-1]/1e3,bend,'ro', markersize=12)
    
    ax2.set_ylabel('Grounding line position (km)')  # we already handled the x-label with ax1
    ax2.set_xlabel('Time (years)')  # we already handled the x-label with ax1
    ax2.plot(time, L/1e3)
    
    ax3.set_ylabel('Fluxes (m^2/yr)')  # we already handled the x-label with ax1
    ax3.set_xlabel('x (km)')  # we already handled the x-label with ax1
    ax3.plot(x, accum*x*1e3,'b')
    ax3.plot(x, omega*(hgs**beta),'r')
    ax3.plot(L[0]/1e3, accum*L[0],'bs', markersize=12)
    ax3.plot(L[0]/1e3, omega*(hgsl[0]**beta),'rs', markersize=12)
    ax3.plot(L[-1]/1e3, accum*L[-1],'bo', markersize=12)
    ax3.plot(L[-1]/1e3, omega*(hgsl[-1]**beta),'ro', markersize=12)
    plt.ylim(0,10e5)
    plt.xlim(500,1600)
    
    plt.show()
    
    plt.show()

## Generate interactive plot of simple model evolution

In [34]:
interactive_plot = interactive(plotmodel, accum=(0.05,0.7,0.05), Linit=(600e3,1500e3,100e3))
interactive_plot

interactive(children=(FloatSlider(value=0.35000000000000003, description='accum', max=0.7, min=0.05, step=0.05…

## Define Flowline Model

In [46]:
def bed(x,params):
    
    #Schoof (2007) bed topo (aka MISMIP bed)
    b = -(729 - 2184.8*(x/750e3)**2 +1031.72*(x/750e3)**4 - 151.72*(x/750e3)**6);

    return b

In [131]:
def flowline_eqns(huxg,*huxglast):

    #oh no, defining all parameters in the function, don't do this at home
    params = {'b0': -100};           #bed topo at x=0
    params['bx'] = -0.001;           #linear bed slope

    params['sill_min'] = 2000e3;   #sill min x position
    params['sill_max'] = 2100e3;   #sill max x position
    params['sill_slope'] = 1e-3;   #slope of sill

    #Physical parameters
    params['year'] = 3600*24*365;  #number of seconds in a year
    params['Aglen'] = 4.227e-25;   #ice softness parameter
    params['nglen'] = 3;           #Glen's exponent
    params['Bglen'] = params['Aglen']**(-1/params['nglen']);
    params['m']     = 1/params['nglen'];  #sliding exponent (power law)
    params['accum'] = 0.28/params['year']; #SMB (constant here)
    params['C']     = 7e6;        #sliding coefficient (power law)
    params['rhoi']  = 917;         #ice density
    params['rhow']  = 1028;        #water density
    params['g']     = 9.81;        #gravity accel

    #Scaling params (coupled model equations solved in non-dim form)
    params['hscale'] = 1000;               #thickness scaling
    params['ascale'] = 0.1/params['year'];    #SMB scaling 
    params['uscale'] = (params['rhoi']*params['g']*params['hscale']*params['ascale']/params['C'])**(1/(params['m']+1)); #velocity scaling
    params['xscale'] = params['uscale']*params['hscale']/params['ascale'];  #horizontal distance scaling
    params['tscale'] = params['xscale']/params['uscale'];                #time scaling
    params['eps']    = params['Bglen']*((params['uscale']/params['xscale'])**(1/params['nglen']))/(2*params['rhoi']*params['g']*params['hscale']);  #epsilon param (Schoof 2007)
    params['lambda'] = 1 - params['rhoi']/params['rhow'];  #density difference (lambda param Schoof 2007)
    params['transient'] = 0;                  #0 if solving for steady-state, 1 if solving for transient evolution

    #Grid parameters
    params['tfinal'] = 10e3*params['year'];   #length of transient simulation
    params['Nt'] = 1e2;                       #number of time steps
    params['dt'] = params['tfinal']/params['Nt'];   #time step length
    params['Nx'] = 200;                       #number of grid points
    params['N1'] = 100;                       #number of grid points in coarse domain
    params['sigGZ'] = 0.97;                   #extent of coarse grid (where GL is at sigma=1)
    sigma1=np.linspace(params['sigGZ']/(params['N1']+0.5), params['sigGZ'], params['N1']);
    sigma2=np.linspace(params['sigGZ'], 1, params['Nx']-params['N1']+1);
    params['sigma'] = np.concatenate((sigma1, sigma2[1:params['Nx']-1]));    #grid points on velocity (includes GL, not ice divide)
    sigma = params['sigma'];
    params['sigma_elem'] = np.concatenate(([0],(sigma[0:params['Nx']-2] + sigma[1:params['Nx']-1])/2)); #grid points on thickness (includes divide, not GL)
    params['dsigma'] = np.diff(params['sigma']); #grid spacing
    
    #vars unpack
    Nx = params['Nx'];
    h = huxg[0:Nx];
    u = huxg[Nx:2*Nx-1];
    xg = huxg[2*Nx];
    
    #grid params unpack
    dt = params['dt']/params['tscale'];
    ds = params['dsigma'];
    N1 = params['N1'];
    sigma = params['sigma'];
    sigma_elem = params['sigma_elem'];
    Fh = np.zeros(Nx);
    Fu = np.zeros(Nx);

    #physical params unpack
    m     = params['m'];
    nglen = params['nglen'];
    lambdap = params['lambda'];
    accum = params['accum'];
    a = accum/params['ascale'];
    eps = params['eps'];
    ss = params['transient'];
    xscale = params['xscale'];
    hscale = params['hscale'];
    
    #previous time step unpack
    b = -1*bed(xg*sigma*xscale,params)/hscale;
    hf = (-1*bed(xg*xscale,params)/hscale)/(1-lambdap);
    h_old, u_old, xg_old = huxglast

    #thickness
    Fh[0]      = ss*(h[0]-h_old[0])/dt + (2*h[0]*u[0])/(ds[0]*xg) - a;
    Fh[1]      = ss*(h[1]-h_old[1])/dt - ss*sigma_elem[1]*(xg-xg_old)*(h[2]-h[0])/(2*dt*ds[1]*xg) + (h[1]*(u[1]+u[0]))/(2*xg*ds[1]) - a;
    Fh[2:Nx-2] = ss*(h[2:Nx-2]-h_old[2:Nx-2])/dt - ss*sigma_elem[2:Nx-2]*(xg-xg_old)*(h[3:Nx-1]-h[1:Nx-3])/(2*dt*ds[2:Nx-2]*xg) + (h[2:Nx-2]*(u[2:Nx-2]+u[1:Nx-3]) - h[1:Nx-3]*(u[1:Nx-3]+u[0:Nx-4]))/(2*xg*ds[2:Nx-2]) - a;

    Fh[N1-1] = (1+0.5*(1+(ds[N1-1])/ds[N1-2]))*h[N1-1] - 0.5*(1+(ds[N1-1]/ds[N1-2]))*h[N1-2] - h[N1];

    Fh[Nx-1]     = ss*(h[Nx-1]-h_old[Nx-1])/dt - ss*sigma_elem[Nx-2]*(xg-xg_old)*(h[Nx-1]-h[Nx-2])/(dt*ds[Nx-2]*xg) + (h[Nx-1]*(u[Nx-1]+u[Nx-2]) - h[Nx-2]*(u[Nx-2]+u[Nx-3]))/(2*xg*ds[Nx-2]) - a;

  #velocity
    Fu[0]      = (4*eps)*(1/(xg*ds[0])**((1/nglen)+1))*(h[1]*(u[1]-u[0])*np.absolute(u[1]-u[0])**((1/nglen)-1) - h[0]*(2*u[0])*np.absolute(2*u[0])**((1/nglen)-1)) - u[0]*np.absolute(u[0])**(m-1) - 0.5*(h[0]+h[1])*(h[1]-b[1]-h[0]+b[0])/(xg*ds[0]);
    Fu[1:Nx-2] = (4*eps)*(1/(xg*ds[1:Nx-2])**((1/nglen)+1))*(h[2:Nx-1]*(u[2:Nx-1]-u[1:Nx-2])*np.absolute(u[2:Nx-1]-u[1:Nx-2])**((1/nglen)-1) -h[1:Nx-2]*(u[1:Nx-2]-u[0:Nx-3])*np.absolute(u[1:Nx-2]-u[0:Nx-3])**((1/nglen)-1)) -u[1:Nx-2]*np.absolute(u[1:Nx-2])**(m-1) -0.5*(h[1:Nx-2]+h[2:Nx-1])*(h[2:Nx-1]-b[2:Nx-1]-h[1:Nx-2]+b[1:Nx-2])/(xg*ds[1:Nx-2]);
    Fu[N1-1]     = (u[N1]-u[N1-1])/ds[N1-1] - (u[N1-1]-u[N1-2])/ds[N1-2];
    Fu[Nx-1]     = (1/(xg*ds[Nx-2])**(1/nglen))*(np.absolute(u[Nx-1]-u[Nx-2])**((1/nglen)-1))*(u[Nx-1]-u[Nx-2]) - lambdap*hf/(8*eps);

    Fxg        = 3*h[Nx-1] - h[Nx-2] - 2*hf;

    F = np.concatenate((Fh,Fu,Fxg));
    return F


## Solve for steady-state initial conditions

In [132]:
params['accum'] = 1/params['year'];
xg = 1500e3/params['xscale'];
hf = (-bed(xg*params['xscale'],params)/params['hscale'])/(1-params['lambda']);
h = 1 - (1-hf)*params['sigma'];
u = 0.3*(params['sigma']**(1/3)) + 1e-3;

huxg0 = np.concatenate((h,u,[xg]));
huxglast = (h,u,xg)
h_old, u_old, xg_old = huxglast
np.size(huxg0)

401

In [133]:
from scipy.optimize import fsolve
root = fsolve(flowline_eqns, huxg0,args=huxglast)

IndexError: index 199 is out of bounds for axis 0 with size 199